In [9]:
import os, random, sys, time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

In [10]:
file = open('input/login.txt')
lines = file.readlines()
username = lines[0]
password = lines[1]

In [30]:
link = 'https://www.linkedin.com/uas/login'
file = open('input/messages.txt')
customMessage = file.read()
file1 = open('input/subject.txt')
subject = file1.read()
print('Subject: ' + subject)
print('Messages: ' + customMessage)
data = pd.read_csv("input/data.csv")
data[:]

Subject: Invitation
Messages: This is Henry Duong, Founder of Global QT-Group with the following two advanced competitive features:
QT-AI (global TradeMark) improves 30% operational productivity. 
QT-Invest (global TradeMark) improves 20% financial productivity.
Let's connect and have a good collaboration!


,name,linkedin_link,facebook_link
0,NaN,https://www.linkedin.com/in/fergalmullen/,NaN
1,NaN,https://www.linkedin.com/in/forest-baskett-a58...,NaN
2,NaN,https://www.linkedin.com/in/peter-barris-7a440...,NaN


In [31]:
def check_exists_class(_class):
    try:
        browser.find_element_by_class_name(_class)
    except NoSuchElementException:
        return False
    return True

In [33]:
#Open link on chrome
browser = webdriver.Chrome('driver/chromedriver')
browser.get(link)
time.sleep(random.uniform(3, 7))

#Access account
elementID = browser.find_element_by_id('username')
elementID.send_keys(username)
elementID = browser.find_element_by_id('password')
elementID.send_keys(password)
elementID.submit()
# time.sleep(random.uniform(3, 7))
print('Login to Linkedin')

#Open file visitedUsers 
visitedUsers = pd.read_csv('output/visited_user.csv')
print('LinkedinBot: START')
for _,row in data.iterrows():
    try:
        full_link = row['linkedin_link']
        if full_link in visitedUsers.values:
            for _,rowVisited in visitedUsers.iterrows():
                if rowVisited['link'] in full_link:
                    if rowVisited['status'] in 'error':
                        print('Error: ', full_link)
                    elif rowVisited['status'] in 'done':
                        print('Already sent to: ', full_link)
                    else:
                        print(rowVisited['status'], full_link)
                    break
        else:
            #Send message
            browser.get(full_link)
            time.sleep(random.uniform(3, 7))

            #Connect
            checkProfileActions = browser.find_element_by_class_name('pv-s-profile-actions')
            if 'pv-s-profile-actions--connect' in checkProfileActions.get_attribute('class').split():
                browser.find_element_by_class_name('pv-s-profile-actions--connect').click()
                browser.find_element_by_xpath("//button[contains(@aria-label, 'Add a note')]").click()
                browser.find_element_by_id('custom-message').send_keys(customMessage)
                checkSentButton = browser.find_element_by_class_name('ml1')
                if 'artdeco-button--disabled' in checkSentButton.get_attribute('class').split():
                    print(0)
                    browser.find_element_by_class_name('artdeco-modal__dismiss').click()
                    browser.find_element_by_class_name('pv-s-profile-actions--message').click()
                    time.sleep(random.uniform(3, 5))
                    if True == check_exists_class('msg-form__subject'):
                        print(1)
                        browser.find_element_by_class_name('msg-form__subject').send_keys(subject)
                    if True == check_exists_class('msg-form__contenteditable'):
                        print(2)
                        browser.find_element_by_class_name('msg-form__contenteditable').send_keys(customMessage)
                    time.sleep(random.uniform(2, 3))
                    if True == check_exists_class('msg-form__send-button'):
                        print(3)
                        browser.find_element_by_class_name('msg-form__send-button').click()
                else:
                    checkSentButton.click()
                    
            # Follow
            elif 'pv-s-profile-actions--follow' in checkProfileActions.get_attribute('class').split():
                browser.find_element_by_class_name('pv-s-profile-actions--follow').click()
                time.sleep(random.uniform(2, 4))
                browser.find_element_by_class_name('pv-s-profile-actions__overflow-toggle').click()
                browser.find_element_by_class_name('pv-s-profile-actions--connect').click()
                time.sleep(random.uniform(3, 7))
                browser.find_element_by_xpath("//button[contains(@aria-label, 'Add a note')]").click()
                browser.find_element_by_id('custom-message').send_keys(customMessage)
                browser.find_element_by_class_name('ml1').click()

            print(full_link, ': sent')
            visitedUsers = visitedUsers.append({'link':full_link, 'status':'done'}, ignore_index = True)
            visitedUsers.to_csv('output/visited_user.csv',index=False) 
            # Otherwise, sleep to make sure everything loads
            time.sleep(random.uniform(3, 7))
            
    except: 
        visitedUsers = visitedUsers.append({'link':full_link, 'status':'error'}, ignore_index = True)
        visitedUsers.to_csv('output/visited_user.csv',index=False) 
        
print('LinkedinBot: DONE!')
        

Login to Linkedin
LinkedinBot: START
0
https://www.linkedin.com/in/fergalmullen/ : sent
LinkedinBot: DONE!
